In [16]:
import subprocess
import ffmpeg
import moviepy.editor as mpe
import rubberband
import googletrans
import gtts
import pyttsx3
import mediapipe

def translate_video(video_file, target_language, output_file):
  """Translates the voice in a video to a target language and generates a lip-synced audio file of the translation.

  Args:
    video_file: The path to the input video file.
    target_language: The target language code.
    output_file: The path to the output audio file.
  """

  # Extract the audio from the video file.
  audio_file = ffmpeg.input(video_file).output("audio.wav", format="wav").run(capture_output=True)[1]

  # Translate the audio file to the target language.
  translator = googletrans.Translator()
  translated_text = translator.translate(audio_file.decode("utf-8"), dest=target_language).text


  # Generate a lip-synced audio file of the translation.
  engine = pyttsx3.Engine()
  engine.setProperty("rate", 150)
  tts = gtts.gTTS(text=translated_text, lang=target_language, voice=engine.getProperty("voice"))
  tts.save(output_file)

  # Warp the translated audio to match the length and tone of the original audio.
  rubberband = rubberband.RubberBand()
  rubberband.set_rate(0.5)
  warped_audio = rubberband.process(output_file)

  # Write the warped audio to a new file.
  with open("warped_audio.wav", "wb") as f:
    f.write(warped_audio)

def replace_audio_in_video(video_file, audio_file, output_file):
  """Replaces the audio in a video with a new audio file.

  Args:
    video_file: The path to the input video file.
    audio_file: The path to the input audio file.
    output_file: The path to the output video file.
  """

  ffmpeg.input(video_file).input(audio_file, ss=0.5).output(output_file, pix_fmt="yuv420p", acodec="aac").run()

if __name__ == "__main__":
  # Get the video file from the user.
  video_file = input("Enter the path to the video file: ")

  # Get the target language from the user.
  target_language = input("Enter the target language: ")

  # Translate the video and generate a lip-synced audio file of the translation.
  translate_video(video_file, target_language, "output.wav")

  # Replace the audio in the video with the translated audio.
  replace_audio_in_video(video_file, "warped_audio.wav", "output_video.mp4")

  # Print a message to the user.
  print("The output video has been generated and saved as `output_video.mp4`.")



Enter the path to the video file: /content/drive/MyDrive/input/VID_20231022_170736.mp4
Enter the target language: english


TypeError: ignored

In [15]:
import mediapipe as mp
import rubberband
import moviepy.editor as mpe
import googletrans
import gtts
import pyttsx3

def translate_video(video_file, target_language, output_file):
  """Translates the voice in a video to a target language and generates a lip-synced audio file of the translation.

  Args:
    video_file: The path to the input video file.
    target_language: The target language code.
    output_file: The path to the output audio file.
  """

  # Extract the audio from the video file.
  video = mpe.VideoFileClip(video_file)
  audio = video.audio

  # Translate the audio file to the target language.
  translator = googletrans.Translator()
  translated_text = translator.translate(audio.read(), dest=target_language).text

  # Generate a lip-synced audio file of the translation.
  engine = pyttsx3.Engine()
  engine.setProperty("rate", 150)
  tts = gTTS(text=translated_text, lang=target_language, voice=engine.getProperty("voice"))
  tts.save("tmp.wav")

  # Create a temporary audio file to save the lip-synced audio to.
  tmp_audio_clip = mpe.AudioFileClip("tmp.wav")

  # Extract the pitch of the original voice in the video.
  with mp.solutions.holistic.Holistic() as holistic:
    for results in holistic.process(video.read_frame()):
      pitch = results.face_landmarks.landmark[mp.solutions.holistic.HolisticLandmark.LEFT_EAR].z * 440

  # Shift the pitch of the translated audio to match the pitch of the original audio.
  shifted_audio_clip = rubberband.RubberBand()
  shifted_audio_clip.set_rate(original_pitch / target_pitch)
  shifted_audio_clip = shifted_audio_clip.process(tmp_audio_clip.read())

  # Replace the audio in the video with the cloned audio.
  video.audio = mpe.AudioSegment(shifted_audio_clip)

  # Write the output video file.
  video.write_videofile(output_file, audiocodec="aac")

  # Delete the temporary audio file.
  tmp_audio_clip.close()
  os.remove("tmp.wav")


if __name__ == "__main__":
  # Get the video file from the user.
  video_file = input("Enter the path to the video file: ")

  # Get the target language from the user.
  target_language = input("Enter the target language: ")

  # Translate the video and generate a lip-synced audio file of the translation.
  translate_video(video_file, target_language, "output.mp4")

  # Print a message to the user.
  print("The output video has been generated and saved as `output.mp4`.")



Enter the path to the video file: /content/drive/MyDrive/input/VID_20231022_170736.mp4
Enter the target language: english


AttributeError: ignored

In [17]:
import mediapipe as mp
import rubberband
import moviepy.editor as mpe
import googletrans
import gtts
import pyttsx3

def transcribe_audio(audio_file_path, language_code="en-US"):
  """Transcribes an audio file to text.

  Args:
    audio_file_path: The path to the audio file.
    language_code: The language code of the audio file.

  Returns:
    A string containing the text transcription of the audio file.
  """

  client = speech_v1.SpeechClient()

  with io.open(audio_file_path, "rb") as audio_file:
    content = audio_file.read()

  audio = speech_v1.types.Audio(content=content)
  config = speech_v1.types.RecognitionConfig(
      language_code=language_code,
      encoding="LINEAR16",
      sample_rate_hertz=16000
  )

  response = client.recognize(config=config, audio=audio)

  if response.results:
    return response.results[0].alternatives[0].transcript
  else:
    return None


def translate_video(video_file, target_language, output_file):
  """Translates the voice in a video to a target language and generates a lip-synced audio file of the translation.

  Args:
    video_file: The path to the input video file.
    target_language: The target language code.
    output_file: The path to the output audio file.
  """

  # Extract the audio from the video file.
  video = mpe.VideoFileClip(video_file)
  audio = video.audio

  # Transcribe the audio to text.
  transcribed_text = transcribe_audio(audio.read())

  # Translate the text to the target language.
  translator = googletrans.Translator()
  translated_text = translator.translate(transcribed_text, dest=target_language).text

  # Generate a lip-synced audio file of the translation.
  engine = pyttsx3.Engine()
  engine.setProperty("rate", 150)
  tts = gTTS(text=translated_text, lang=target_language, voice=engine.getProperty("voice"))
  tts.save("tmp.wav")

  # Create a temporary audio file to save the lip-synced audio to.
  tmp_audio_clip = mpe.AudioFileClip("tmp.wav")

  # Extract the pitch of the original voice in the video.
  with mp.solutions.holistic.Holistic() as holistic:
    for results in holistic.process(video.read_frame()):
      pitch = results.face_landmarks.landmark[mp.solutions.holistic.HolisticLandmark.LEFT_EAR].z * 440

  # Shift the pitch of the translated audio to match the pitch of the original audio.
  shifted_audio_clip = rubberband.RubberBand()
  shifted_audio_clip.set_rate(original_pitch / target_pitch)
  shifted_audio_clip = shifted_audio_clip.process(tmp_audio_clip.read())

  # Replace the audio in the video with the cloned audio.
  video.audio = mpe.AudioSegment(shifted_audio_clip)

  # Write the output video file.
  video.write_videofile(output_file, audiocodec="aac")

  # Delete the temporary audio file.
  tmp_audio_clip.close()
  os.remove("tmp.wav")


if __name__ == "__main__":
  # Get the video file from the user.
  video_file = input("Enter the path to the video file: ")

  # Get the target language from the user.
  target_language = input("Enter the target language: ")

  # Translate the video and generate a lip-synced audio file of the translation.
  translate_video(video_file, target_language, "output.mp4")

  # Print a message to the user.
  print("The output video has been generated and saved as `output.mp4`.")




Enter the path to the video file: /content/drive/MyDrive/input/VID_20231022_170736.mp4
Enter the target language: english


AttributeError: ignored

In [13]:
pip install moviepy --upgrade

In [1]:
import mediapipe as mp
import rubberband
import moviepy.editor as mpe
import googletrans
import gTTS
import pyttsx3

def translate_video(video_file, target_language, output_file):
  """Translates the voice in a video to a target language and generates a lip-synced audio file of the translation.

  Args:
    video_file: The path to the input video file.
    target_language: The target language code.
    output_file: The path to the output audio file.
  """

  # Extract the audio from the video file.
  video = mpe.VideoFileClip(video_file)
  audio = video.audio

  # Translate the audio file to the target language.
  translator = googletrans.Translator()
  translated_text = translator.translate(audio.data, dest=target_language).text

  # Generate a lip-synced audio file of the translation.
  engine = pyttsx3.Engine()
  engine.setProperty("rate", 150)
  tts = gTTS(text=translated_text, lang=target_language, voice=engine.getProperty("voice"))
  tts.save("tmp.wav")

  # Create a temporary audio file to save the lip-synced audio to.
  tmp_audio_clip = mpe.AudioFileClip("tmp.wav")

  # Extract the pitch of the original voice in the video.
  with mp.solutions.holistic.Holistic() as holistic:
    for results in holistic.process(video.read_frame()):
      pitch = results.face_landmarks.landmark[mp.solutions.holistic.HolisticLandmark.LEFT_EAR].z * 440

  # Shift the pitch of the translated audio to match the pitch of the original audio.
  shifted_audio_clip = rubberband.RubberBand()
  shifted_audio_clip.set_rate(original_pitch / target_pitch)
  shifted_audio_clip = shifted_audio_clip.process("tmp.wav")

  # Replace the audio in the video with the cloned audio.
  video.audio = shifted_audio_clip

  # Write the output video file.
  video.write_videofile(output_file, audiocodec="aac")

  # Delete the temporary audio file.
  tmp_audio_clip.close()
  os.remove("tmp.wav")


if __name__ == "__main__":
  # Get the video file from the user.
  video_file = input("Enter the path to the video file: ")

  # Get the target language from the user.
  target_language = input("Enter the target language: ")

  # Translate the video and generate a lip-synced audio file of the translation.
  translate_video(video_file, target_language, "output.mp4")

  # Print a message to the user.
  print("The output video has been generated and saved as `output.mp4`.")



ModuleNotFoundError: ignored

In [2]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 43.0 MB/s eta 0:00:00


In [3]:
pip install ffmpeg-python

In [3]:
pip install pyttsx3

In [4]:
pip install gtts

In [5]:
pip install googletrans==4.0.0rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.4 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=38adf4af18e055a8dc4705ebf313784fe0d0a578c3570e19e37a7a2c64653028
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling 

In [6]:
!apt-get install -y build-essential cmake ninja-build pkg-config

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.1).
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 0s (253 kB/s)
Selecting previously unselected package ninja-build.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../ninja-build_1.10.1-1_amd64.deb ...
Unpacking ninja-build (1.10.1-1) ...
Setting up ninja-build (1.10.1-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [7]:
pip install moviepy

In [12]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [589 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,012 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,278 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,185 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packag

In [8]:
!apt-get install -y libsndfile-dev librubberband-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libsndfile1-dev' instead of 'libsndfile-dev'
libsndfile1-dev is already the newest version (1.0.31-2build1).
The following NEW packages will be installed:
  librubberband-dev libsamplerate0-dev
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 189 kB of archives.
After this operation, 743 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsamplerate0-dev amd64 0.2.2-1build1 [61.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 librubberband-dev amd64 2.0.0-2 [127 kB]
Fetched 189 kB in 1s (337 kB/s)
Selecting previously unselected package libsamplerate0-dev:amd64.
(Reading database ... 120886 files and directories currently installed.)
Preparing to unpack .../libsamplerate0-dev_0.2.2-1build1_amd64.deb ...
Unpacking libsamplerate0-dev:amd64 (0.2.2-1build1) ...
Selecting previously unse

In [14]:
pip install rubberband

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for rubberband
  Running setup.py clean for rubberband
Failed to build rubberband
ERROR: Could not build wheels for rubberband, which is required to install pyproject.toml-based projects


In [ ]:
pip install gdown

In [ ]:
!gdown https://github.com/RubberBand/rubberband/archive/refs/heads/main.zip

Downloading...
From: https://github.com/RubberBand/rubberband/archive/refs/heads/main.zip
To: /content/main.zip
 31% 9.00/29.0 [00:00<00:00, 32.7kB/s]


In [ ]:
!unzip main.zip

Archive:  main.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of main.zip or
        main.zip.zip, and cannot find main.zip.ZIP, period.


In [9]:
!unzip /content/drive/MyDrive/input/rubberband-1.0.2.zip

Archive:  /content/drive/MyDrive/input/rubberband-1.0.2.zip
   creating: rubberband-1.0.2/
   creating: rubberband-1.0.2/ext/
  inflating: rubberband-1.0.2/ext/deepClean.py  
  inflating: rubberband-1.0.2/ext/__init__.py  
  inflating: rubberband-1.0.2/LICENSE.rst  
  inflating: rubberband-1.0.2/MANIFEST.in  
  inflating: rubberband-1.0.2/PKG-INFO  
  inflating: rubberband-1.0.2/README.rst  
   creating: rubberband-1.0.2/rubberband.egg-info/
  inflating: rubberband-1.0.2/rubberband.egg-info/dependency_links.txt  
  inflating: rubberband-1.0.2/rubberband.egg-info/entry_points.txt  
  inflating: rubberband-1.0.2/rubberband.egg-info/PKG-INFO  
  inflating: rubberband-1.0.2/rubberband.egg-info/SOURCES.txt  
  inflating: rubberband-1.0.2/rubberband.egg-info/top_level.txt  
  inflating: rubberband-1.0.2/rubberband.egg-info/zip-safe  
  inflating: rubberband-1.0.2/setup.cfg  
  inflating: rubberband-1.0.2/setup.py  
   creating: rubberband-1.0.2/src/
  inflating: rubberband-1.0.2/src/Debug.cp

In [10]:
cd rubberband-1.0.2

/content/rubberband-1.0.2


In [9]:
pip install setup.py bdist_wheel --universal


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --universal


In [11]:
pip install .

Processing /content/rubberband-1.0.2
  Preparing metadata (setup.py) ... done
  Created wheel for rubberband: filename=rubberband-1.0.2-cp310-cp310-linux_x86_64.whl size=346232 sha256=0f255f7e1ac1d35c04cc2f499e0107ff718dcb91178cc6ac322c843c2ccff959
  Stored in directory: /root/.cache/pip/wheels/e3/94/16/545c0fe0880d74d38ea5803d030e70feeea935c8c9bd59ef75
Successfully built rubberband
